In [ ]:


from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [3]:

# os for file management*

import os

os.chdir('My Drive')
os.chdir('Colab Notebooks')

In [ ]:
 

!pip install tweepy


In [5]:
# Libs


import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

import pandas as pd

import json
from collections import OrderedDict

In [6]:
# Twitter APIs



consumer_key = 'd8RxF1zqe1vqZGPTFNOX865B2'
consumer_secret = 'B8vjEkOLyirvoBf4zp2cBPAKoSvjGxRbzK0RQgB65OGpmkyeZL'
access_token = '895405449620672513-NAhckCrY8XzAtjtuyUv5CGjYN9pG29r'
access_secret = 'Hf6HSPpP3Mbh6O5xLqwcTMrawSxgQ5lxf9lngisrKsdze'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
user = api.me()

print(user)

In [ ]:
print('Name: ' + user.name)
print('Description: ' + user.description)
print('Friends: ' + str(user.friends_count))
print('Nick: @' + str(user.screen_name) + "  <---- Beni Ekleyebilirsiniz")

In [ ]:
api.home_timeline()

In [ ]:
OrderedDict(api.home_timeline()[1]._json)

In [ ]:
for status in api.home_timeline():
   print(status.text)

In [12]:
tweets = []

for status in api.home_timeline():
   tweets.append(status.text)
  

In [ ]:
len(tweets)

In [ ]:
tweets = []

for status in tweepy.Cursor(api.home_timeline).items(50):
    tweets.append(status.text)
    
len(tweets)

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    print(status.text)

In [ ]:
for status in tweepy.Cursor(api.search, q=('DEDAŞ')).items(10):
  print(status.text)

In [ ]:
# retweet'leri ve yabancı tweet'leri kaldıralım
for status in tweepy.Cursor(api.search, q=('DEDAŞ-filter:retweets'), lang="tr").items(10):
  print(status.text)

In [ ]:
for status in tweepy.Cursor(api.search, q=[ 'elektrik' ,  "DEDAŞ "]).items(10):
  print(status.text)

In [ ]:
for status in tweepy.Cursor(api.search, q=[ "elektrik kesik -filter:retweets"], tweet_mode = "extended").items(1000):
  print(status.full_text)

In [ ]:
OrderedDict(api.home_timeline()[1]._json)

In [ ]:
for status in tweepy.Cursor(api.search, q=('Dedaş elektrik -filter:retweets' or 'Dedaş Kesinti -filter:retweets'), location = "Diyarbakır").items(10):
  print(status.text, status.created_at, status._json["entities"]["hashtags"][0]["text"]   if len(status._json["entities"]["hashtags"])> 0 else None ,status.user.screen_name, status.user.location, '\t', sep = '\n',  )

In [22]:
tweetid = []
tweets = []
ttime = []
hashtags = []
nick = []
location = []


for status in tweepy.Cursor(api.search, q=('Dedaş elektrik' or 'Dedaş Kesinti'), tweet_mode="extended").items(5000):
  tweetid.append(status.id_str)
  tweets.append(status.full_text.lower() )
  ttime.append(status.created_at)
  hashtags.append (status._json["entities"]["hashtags"][0]["text"]   if len(status._json["entities"]["hashtags"])> 0 else None)
  nick.append(status.user.screen_name)
  location.append(status.user.location)
  
Tweetdfread = pd.DataFrame ({"tweetid" : tweetid,
                         'tweets' : tweets,
                         'ttime' : ttime,
                         'hashtags' : hashtags,
                         'nick' : nick,
                         'location' : location
    
})


In [ ]:
Tweetdfread.head()

In [ ]:
Tweetdfread.describe()

In [25]:
Tweetdfread.to_csv( 'KBTweet1.csv',sep=';', index = False)

In [26]:
class SimpleListener(tweepy.StreamListener):
    
    def on_status(self, status):
        #code to run each time you receive some data (direct message, delete, profile update, status,...)
        print(status.text)
    
    def on_timeout(self):
        print(sys.stderr, 'Timeout...')
        return True # Don't kill the stream


In [27]:
myStreamListener = SimpleListener()


In [ ]:
#Canlı veri çekme
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

myStream.filter(track=['kesinti'])